## AirBnB Optimal Price Predictor

This notebook downloads the data from remote used in this project and then cleans it. Ready to be used to build a model for predictions.


In [1]:
# Imports
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import glob
import os
import httplib2
from bs4 import BeautifulSoup, SoupStrainer
from pandas_profiling import ProfileReport

In [2]:
# First Trying to get all the data from a remote site
http = httplib2.Http()
status, response = http.request('http://insideairbnb.com/get-the-data.html')

url_list = []
# loop through the response and get all the links
for link in BeautifulSoup(response, parse_only=SoupStrainer('a')):
    if link.has_attr('href'):
        # Add to the list
        url_list.append(link['href'])

# Show list
# Uncomment to see
# url_list


In [3]:
df = pd.DataFrame(url_list, columns=['URL'])
df

,URL
0,index.html
1,about.html
2,behind.html
3,get-the-data.html
4,https://twitter.com/share
...,...
7683,http://data.insideairbnb.com/switzerland/z%C3%...
7684,http://data.insideairbnb.com/switzerland/z%C3%...
7685,http://data.insideairbnb.com/switzerland/z%C3%...
7686,http://data.insideairbnb.com/switzerland/z%C3%...


In [4]:
# only filter rows that ends with 'listings.csv' and save to new df
df_listings = df[df.URL.str.endswith('listings.csv', na=False)]
df_listings.reset_index(inplace=True)
df_listings

,index,URL
0,14,http://data.insideairbnb.com/the-netherlands/n...
1,21,http://data.insideairbnb.com/the-netherlands/n...
2,28,http://data.insideairbnb.com/the-netherlands/n...
3,35,http://data.insideairbnb.com/the-netherlands/n...
4,42,http://data.insideairbnb.com/the-netherlands/n...
...,...,...
1076,7655,http://data.insideairbnb.com/switzerland/z%C3%...
1077,7662,http://data.insideairbnb.com/switzerland/z%C3%...
1078,7669,http://data.insideairbnb.com/switzerland/z%C3%...
1079,7676,http://data.insideairbnb.com/switzerland/z%C3%...


In [5]:
# build a dataframe with rightly described country, area, city and date of latest data available online
df_listings['country'] = df_listings['URL'].apply(lambda x: x.split('/')[3])
df_listings['area'] = df_listings['URL'].apply(lambda x: x.split('/')[4])
df_listings['city'] = df_listings['URL'].apply(lambda x: x.split('/')[5])
df_listings['date_of_latest_data'] = df_listings['URL'].apply(lambda x: x.split('/')[6])
df_listings


/Users/rob/opt/anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/rob/opt/anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/rob/opt/anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a co

,index,URL,country,area,city,date_of_latest_data
0,14,http://data.insideairbnb.com/the-netherlands/n...,the-netherlands,north-holland,amsterdam,2021-04-09
1,21,http://data.insideairbnb.com/the-netherlands/n...,the-netherlands,north-holland,amsterdam,2021-03-04
2,28,http://data.insideairbnb.com/the-netherlands/n...,the-netherlands,north-holland,amsterdam,2021-02-08
3,35,http://data.insideairbnb.com/the-netherlands/n...,the-netherlands,north-holland,amsterdam,2021-01-09
4,42,http://data.insideairbnb.com/the-netherlands/n...,the-netherlands,north-holland,amsterdam,2020-12-12
...,...,...,...,...,...,...
1076,7655,http://data.insideairbnb.com/switzerland/z%C3%...,switzerland,z%C3%BCrich,zurich,2020-11-29
1077,7662,http://data.insideairbnb.com/switzerland/z%C3%...,switzerland,z%C3%BCrich,zurich,2020-10-29
1078,7669,http://data.insideairbnb.com/switzerland/z%C3%...,switzerland,z%C3%BCrich,zurich,2020-06-25
1079,7676,http://data.insideairbnb.com/switzerland/z%C3%...,switzerland,z%C3%BCrich,zurich,2020-05-30


In [6]:
# Export data to a text file

np.savetxt(r'URL_List_Listings_CSV.txt', df_listings.URL, fmt="%s")


In [7]:
# Run a loop to download all the data in the raw directory below
# ├── data
# │   └── raw            <- The original, immutable data dump.

# Uncomment to execute. Downloads 1.75 GB data from remote
# for i in range(len(df_listings['URL'])):
#     # Loop through each loop
#     url = df_listings['URL'][i]
#     print('Downloading File ', i)
#     print(url, '\n')
#
#     # Get file name for better description of file
#     file_name = df_listings['country'][i]+'_'+df_listings['city'][i]+'_'+df_listings['date_of_latest_data'][i]+'.csv'
#
#     # Get the file
#     file = requests.get(url)
#
#     # Open to write, which saves the file instead
#     open('../data/raw/'+file_name, 'wb').write(file.content)

In [8]:
print(os.getcwd())


/Users/rob/G_Drive_sing.parvi/Colab_Notebooks/DS-Unit-4-BW/ptpt-airbnb-2/notebooks


In [9]:
# Change the directory to raw data
os.chdir(os.path.dirname('../data/raw/'))
print(os.getcwd())

/Users/rob/G_Drive_sing.parvi/Colab_Notebooks/DS-Unit-4-BW/ptpt-airbnb-2/data/raw


In [10]:
# create a list of all csv files
globbed_files = glob.glob('*.csv')

data = []
for csv in globbed_files:
    frame = pd.read_csv(csv)
    frame['filename'] = os.path.basename(csv)
    data.append(frame)
    print(csv," ", 'import complete.')

# Save the data list as a DataFrame
merged_df = pd.concat(data, ignore_index=True)

# Display shape
merged_df


portugal_porto_2020-10-21.csv   import complete.
united-states_salem-or_2020-12-23.csv   import complete.
switzerland_vaud_2020-06-21.csv   import complete.
italy_rome_2020-09-13.csv   import complete.
united-states_jersey-city_2021-03-18.csv   import complete.
canada_new-brunswick_2020-06-25.csv   import complete.
singapore_singapore_2020-05-27.csv   import complete.
singapore_singapore_2021-03-25.csv   import complete.
canada_quebec-city_2021-03-04.csv   import complete.
united-states_san-francisco_2021-02-04.csv   import complete.
united-states_clark-county-nv_2020-05-26.csv   import complete.
italy_bologna_2021-04-20.csv   import complete.
united-kingdom_greater-manchester_2020-10-25.csv   import complete.
canada_quebec-city_2020-07-17.csv   import complete.
portugal_porto_2020-05-28.csv   import complete.
greece_thessaloniki_2020-10-25.csv   import complete.
mexico_mexico-city_2020-06-20.csv   import complete.
spain_menorca_2021-03-30.csv   import complete.
united-states_hawaii_20

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,filename,<html>,number_of_reviews_ltm
0,41339.0,Porto city flat near the sea,180050.0,Paula,PORTO,Lordelo do Ouro e Massarelos,41.15010,-8.66035,Entire home/apt,75.0,5.0,17.0,2018-09-28,0.19,1.0,0.0,portugal_porto_2020-10-21.csv,NaN,NaN
1,42462.0,Large Penthouse in Vila Nova Gaia,185492.0,Gabriela,VILA NOVA DE GAIA,Mafamude e Vilar do Paraíso,41.10739,-8.59430,Entire home/apt,73.0,30.0,39.0,2018-10-14,0.41,3.0,1.0,portugal_porto_2020-10-21.csv,NaN,NaN
2,55111.0,Fontielas Houses |Floor1 in House with shared ...,259711.0,Isabel E João,PAREDES,Cete,41.17481,-8.35362,Entire home/apt,66.0,3.0,14.0,2019-09-07,0.22,3.0,0.0,portugal_porto_2020-10-21.csv,NaN,NaN
3,73828.0,Fontielas Houses |Floor0 in House with shared ...,259711.0,Isabel E João,PAREDES,Cete,41.17449,-8.35426,Entire home/apt,66.0,3.0,8.0,2019-07-22,0.12,3.0,0.0,portugal_porto_2020-10-21.csv,NaN,NaN
4,86388.0,A Formosa - a nest made for two,185209.0,João,PORTO,"Cedofeita, Ildefonso, Sé, Miragaia, Nicolau, V...",41.14918,-8.60922,Private room,20.0,2.0,136.0,2020-09-23,1.18,6.0,231.0,portugal_porto_2020-10-21.csv,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12176730,43348647.0,Recámara Privada,175669735.0,Josue,NaN,Hennepin,45.01486,-93.30547,Private room,40.0,1.0,3.0,2020-05-11,3.00,1.0,362.0,united-states_twin-cities-msa_2020-05-12.csv,NaN,NaN
12176731,43389505.0,Private Room in Private Basement,129740000.0,Aj,NaN,Hennepin,45.00723,-93.29911,Private room,50.0,2.0,0.0,NaN,NaN,1.0,86.0,united-states_twin-cities-msa_2020-05-12.csv,NaN,NaN
12176732,43390452.0,The Grace Space,33383865.0,Andrea,NaN,Ramsey,45.08582,-93.02009,Entire home/apt,159.0,5.0,0.0,NaN,NaN,2.0,87.0,united-states_twin-cities-msa_2020-05-12.csv,NaN,NaN
12176733,43405075.0,Prior Lake Cottage home on prime Bay of the Lake!,111948008.0,Ann,NaN,Scott,44.74567,-93.39000,Entire home/apt,300.0,5.0,0.0,NaN,NaN,1.0,356.0,united-states_twin-cities-msa_2020-05-12.csv,NaN,NaN


In [11]:
print('Before dropping rows the shape of the dataframe is ', merged_df.shape)

# Dropping Columns that may not be used in model or that have high NaN values
merged_df.drop(labels=['id', 'name', 'host_id', 'host_name', 'number_of_reviews_ltm', 'neighbourhood_group', 'reviews_per_month', 'last_review', '<html>'], axis=1, inplace=True)

# Drop Duplicate Values and only retain the later one
merged_df.drop_duplicates(subset=['neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'calculated_host_listings_count',
       'availability_365', 'filename'], keep='last', inplace=True)

# Dropping na rows, only 71 rows
merged_df.dropna(inplace=True)

# Dropping rows where price was too less (less than 10 in local currency) and too high (more than 1 Million in local currency)
merged_df.drop(merged_df[merged_df['price'] < 10].index, inplace=True)
merged_df.drop(merged_df[merged_df['price'] > 1000000].index, inplace=True)

# Dropping rows where minimum_nights is unjustifiable like more than 1500 nights
merged_df.drop(merged_df[merged_df['minimum_nights'] > 1500].index, inplace=True)
# check updated shape
print('After dropping rows the shape of the dataframe is ', merged_df.shape)

Before dropping rows the shape of the dataframe is  (12176735, 19)
After dropping rows the shape of the dataframe is  (12155794, 10)


In [12]:
# Display merged data after Dropping columns
merged_df.head(5)

,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,calculated_host_listings_count,availability_365,filename
0,Lordelo do Ouro e Massarelos,41.15010,-8.66035,Entire home/apt,75.0,5.0,17.0,1.0,0.0,portugal_porto_2020-10-21.csv
1,Mafamude e Vilar do Paraíso,41.10739,-8.59430,Entire home/apt,73.0,30.0,39.0,3.0,1.0,portugal_porto_2020-10-21.csv
2,Cete,41.17481,-8.35362,Entire home/apt,66.0,3.0,14.0,3.0,0.0,portugal_porto_2020-10-21.csv
3,Cete,41.17449,-8.35426,Entire home/apt,66.0,3.0,8.0,3.0,0.0,portugal_porto_2020-10-21.csv
4,"Cedofeita, Ildefonso, Sé, Miragaia, Nicolau, V...",41.14918,-8.60922,Private room,20.0,2.0,136.0,6.0,231.0,portugal_porto_2020-10-21.csv


In [13]:
# Export to the CSV file for future use
merged_df.to_csv('merged_df.csv', index=False)

In [14]:
merged_df1 = merged_df.drop(labels='filename', axis=1)
# Check if the data is having no null values
merged_df1.isnull().sum()

neighbourhood                     0
latitude                          0
longitude                         0
room_type                         0
price                             0
minimum_nights                    0
number_of_reviews                 0
calculated_host_listings_count    0
availability_365                  0
dtype: int64

In [15]:
# Display the summary of statistics
merged_df1.describe(include='all')

,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,calculated_host_listings_count,availability_365
count,12155794,1.215579e+07,1.215579e+07,12155794,1.215579e+07,1.215579e+07,1.215579e+07,1.215579e+07,1.215579e+07
unique,7044,NaN,NaN,4,NaN,NaN,NaN,NaN,NaN
top,I Centro Storico,NaN,NaN,Entire home/apt,NaN,NaN,NaN,NaN,NaN
freq,151180,NaN,NaN,8419984,NaN,NaN,NaN,NaN,NaN
mean,NaN,3.002005e+01,5.946818e-01,NaN,9.141725e+02,6.552030e+00,2.474396e+01,1.378731e+01,1.722744e+02
std,NaN,2.762039e+01,7.490316e+01,NaN,9.022593e+03,2.381185e+01,5.241428e+01,5.168367e+01,1.460285e+02
min,NaN,-4.690624e+01,-1.762681e+02,NaN,1.000000e+01,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00
25%,NaN,3.121871e+01,-5.840575e+01,NaN,6.000000e+01,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00
50%,NaN,4.041108e+01,2.565030e+00,NaN,1.080000e+02,2.000000e+00,4.000000e+00,2.000000e+00,1.680000e+02
75%,NaN,4.554059e+01,1.829750e+01,NaN,2.510000e+02,4.000000e+00,2.300000e+01,6.000000e+00,3.400000e+02


In [16]:
# Export to html using pandas profiling to see the data statistics
profile = ProfileReport(merged_df1, minimal=True)

profile.to_file('Pandas_Profiling_Report.html')



Summarize dataset:   0%|          | 0/18 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]